In [ ]:
!pip install facenet_pytorch

In [2]:
import numpy as np
import torch
import cv2
from matplotlib import pyplot as plt

In [3]:
import os
import random
def get_random_index(start, end, n, seed = -1):
    if seed == -1:
        random.seed()
    else:
        random.seed(seed)
    return sorted(random.sample(range(start, end), n))
    

In [5]:
# generate embeddings
from facenet_pytorch import MTCNN, InceptionResnetV1

mtcnn = MTCNN(keep_all=True)

# load resnet from pretrained model 20180402-114759-vggface2.pt
resnet = InceptionResnetV1(pretrained='vggface2').eval()

/home/himanshi/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
def block_based_warping(image, block_size, key, maximum_pixel_offset, k_size = 15):
    # Create regular grid of blocks
    if block_size == 0:
        warped_image = image.copy()
    else:
        rows, cols = image.shape[:2]
        block_rows, block_cols = rows // block_size, cols // block_size

        # Initialize the warped output image
        warped_image = np.zeros_like(image)

        # Set random seed based on the key
        np.random.seed(key)

        for block_row in range(block_rows):
            for block_col in range(block_cols):
                # Get block indices
                start_row = block_row * block_size
                end_row = start_row + block_size
                start_col = block_col * block_size
                end_col = start_col + block_size

                # Calculate random pixel offsets
                row_offset = np.random.randint(-maximum_pixel_offset, maximum_pixel_offset)
                col_offset = np.random.randint(-maximum_pixel_offset, maximum_pixel_offset)

                # Calculate the warped grid for the block
                warped_start_row = max(0, start_row + row_offset)
                warped_end_row = min(rows, end_row + row_offset)
                warped_start_col = max(0, start_col + col_offset)
                warped_end_col = min(cols, end_col + col_offset)

                # Perform spline interpolation to warp the block
                warped_block = cv2.resize(image[warped_start_row:warped_end_row, warped_start_col:warped_end_col],
                                        (block_size, block_size))

                # Assign the warped block to the corresponding region in the output image
                warped_image[start_row:end_row, start_col:end_col] = warped_block
                
    # Apply Gaussian blur to smoothen the output image
    if k_size != 0:
        kernel_size = (k_size, k_size)  # Increase the kernel size for more smoothing
        smoothed_image = cv2.GaussianBlur(warped_image, kernel_size, 0)
    else:
        smoothed_image = warped_image
    return smoothed_image

In [7]:
# # Setting the input parameters
# image = cv2.imread('ffhq256_pp/test/images/69001.png', cv2.IMREAD_COLOR)
# block_size = 20
# key = 1234
# maximum_pixel_offset = 3

# # Calling the warping function
# warped_image = block_based_warping(image, block_size, key, maximum_pixel_offset)

# # Displaying the original and warped images
# plt.imshow(image)
# plt.show()
# plt.imshow(warped_image)
# plt.show()

In [8]:
# random.seed(0)
# print(get_random_index(0, 100, 5, 0))

In [10]:
# function for above code
def generate_warped_dataset(image_data_path, samples_index, target_folder, block_size, maximum_pixel_offset, k_size, key):
    for i in samples_index:
        img = cv2.imread(image_data_path + "/{:05d}.png".format(i))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        # warp image using block based warping
        warped_image = block_based_warping(img, block_size, key, maximum_pixel_offset, k_size)

        # detect face in the unwarped image and save it
        mtcnn(img, save_path = target_folder + "/original_images/{:05d}.png".format(i))

        # detect face in the warped image and save it
        img_cropped = mtcnn(warped_image, save_path = target_folder + "/images/{:05d}.png".format(i))

        # if no face detected, remove the image OR if multiple faces detected, remove the extra faces
        if img_cropped is None:
            print("No face detected in image: ", i)
            continue
        if img_cropped.shape[0] > 1:
            for j in range(1, img_cropped.shape[0]):
                os.remove(target_folder + "/images/{:05d}_{}.png".format(i,j+1))

            img_cropped = img_cropped[0]

        # generate embedding
        img_cropped = img_cropped.reshape(1,3, 160, 160)
        try:
            img_embedding = resnet(img_cropped)
        except:
            print('Error in processing image:',i)
            # remove the image copy
            os.remove(target_folder + "/images/{:05d}.png".format(i))
            continue

        embedding = img_embedding.detach().numpy()
        file_name = target_folder + "/embeddings/{:05d}.npy".format(i)
        np.save(file_name, embedding)
    



Creating folder:  warped_dataset/8_4_15
No face detected in image:  69081
No face detected in image:  69090


In [ ]:
# params
read_folder = "datasets/ffhq256"
block_size = 8
maximum_pixel_offset = 4
k_size = 15
key = 1234
seed = 0

# create save folder
save_folder = "warped_dataset/{}_{}_{}".format(block_size, maximum_pixel_offset, k_size)

if not os.path.exists(save_folder):
    print("Creating folder: ", save_folder)
    os.makedirs(save_folder)

# create sub folder for train and test
if not os.path.exists(save_folder + "/train/images"):
    os.makedirs(save_folder + "/train/images")
    os.makedirs(save_folder + "/train/embeddings")
    os.makedirs(save_folder + "/train/original_images")
if not os.path.exists(save_folder + "/test/images"):
    os.makedirs(save_folder + "/test/images")
    os.makedirs(save_folder + "/test/embeddings")
    os.makedirs(save_folder + "/test/original_images")

# sample 1000 images for train and test
train_samples = get_random_index(0,68999,1000,seed)
test_samples = range(69000, 70000)

# generate warped dataset
generate_warped_dataset(read_folder, train_samples, save_folder + "/train", block_size, maximum_pixel_offset, k_size, key)
generate_warped_dataset(read_folder, test_samples, save_folder + "/test", block_size, maximum_pixel_offset, k_size, key)

In [18]:
# compare the embeddings

# load any warped image embedding
emb = np.load("warped_dataset/20_3_9/train/embeddings/00009.npy")

# load the original image embedding
emb2 = np.load("datasets/ffhq256_pp/train/embeddings/00009.npy")

norm = np.linalg.norm(emb - emb2)
print(norm)

0.6384911
